In [1]:
# Cellar.ai Wine Recommendation Prototype
# Save this as: wine_recommendation_prototype.ipynb
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

2025-06-30 19:28:24.571756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 19:28:24.580877: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 19:28:24.652093: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 19:28:24.709088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751275704.764911  819514 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751275704.78

In [2]:
def extract_year(text):
    match = re.search(r'\b(\d{4})\b', str(text))
    if match:
        return int(match.group(1))
    return None


def categorize_wine_type(variety):
    
    # Convert to lowercase for easier matching
    variety_lower = variety.lower()
    
    # Reds
    red_varieties = [
        'shiraz', 'cabernet sauvignon', 'pinot noir', 'grenache', 'syrah', 
        'sangiovese', 'petite sirah', 'merlot', 'mataro', 'zinfandel', 
        'durif', 'mourvèdre', 'red blend', 'rhône-style red blend',
        'bordeaux-style red blend', 'g-s-m', 'cabernet blend'
    ]
    
    # Whites  
    white_varieties = [
        'viognier', 'riesling', 'chardonnay', 'sémillon', 'semillon',
        'pinot gris', 'pinot grigio', 'sauvignon blanc', 'sauvignon',
        'roussanne', 'marsanne', 'white blend'
    ]
    
    # Sparkling
    sparkling_varieties = [
        'sparkling blend', 'champagne blend'
    ]
    
    # Rosé
    rose_varieties = [
        'rosé'
    ]
    
    # Check for sparkling first
    if any(spark in variety_lower for spark in sparkling_varieties):
        return 'sparkling'
    
    # Check for rosé
    if any(rose in variety_lower for rose in rose_varieties):
        return 'rosé'
    
    # Check for reds (including blends with red grapes)
    if any(red in variety_lower for red in red_varieties):
        return 'red'
    
    # Check for whites
    if any(white in variety_lower for white in white_varieties):
        return 'white'
    
    # Sweet wines
    if 'moscato' in variety_lower:
        return 'other'
    
    # Everything else
    return 'other'


In [3]:
# ===============================
# 0. CONSTS
# ===============================

vintage_min = 1985
vintage_max = 2025

In [4]:
# ===============================
# 1. LOAD WINE MAG
# ===============================


df = pd.merge(pd.read_csv('../data/winemag-data_first150k.csv'), pd.read_csv('../data/winemag-data-130k-v2.csv'), how='outer')


df = df.drop(['taster_name','taster_twitter_handle'], axis=1)

# Clean the data
df = df[df['country'].isin(['Australia'])]

# df['price'] = df['price'].fillna(df['price'].median())

df['year'] = df['description'].apply(extract_year)
df = df.dropna(subset=['year'])
df['year'] = df['year'].astype(int)

# Categorize

df['variety'] = df['variety'].str.lower()

df['type'] = df['variety'].apply(categorize_wine_type)

# Cleanups

scaler = MinMaxScaler()
df['rank'] = scaler.fit_transform(df[['points']])


df['type'] = df['variety'].apply(categorize_wine_type)


df['description'] = df['description'].str.lower()

print(f"✅ Loaded {len(df)} wines from winemag file")



✅ Loaded 2425 wines from winemag file


In [5]:
# ===============================
# 2. LOAD VIVINO  
# ===============================

# Winery,Year,Wine_ID,Wine,Rating,Reviews,Price,Region,Primary_Grape,Natural,Country,Style,Country_Code

df_vv = pd.read_csv('../data/vivino_top_ten.csv')
df_vv.columns = df_vv.columns.str.lower()
df_vv = df_vv.rename(columns={'primary_grape': 'variety', 'style': 'type', 'wine': 'description'})

df_vv['type'] = df_vv['type'].str.lower()
df_vv['year'] = pd.to_numeric(df_vv['year'], errors='coerce')
df_vv = df_vv.dropna(subset=['year'])
df_vv['year'] = df_vv['year'].astype(int)

# Rank
scaler = MinMaxScaler()
df_vv['rank'] = scaler.fit_transform(df_vv[['rating']])

print(f"\n📊 Vivino Dataset Overview:")
print(f"   Total wines: {len(df_vv):,}")
print(f"   Columns: {list(df_vv.columns)}")
print(f"   Price range: ${df_vv['price'].min():.0f} - ${df_vv['price'].max():.0f}")
# print(f"   Regions: {list(df['province'].drop_duplicates())}")
print(f"   Varietal: {list(df_vv['variety'].drop_duplicates())}")
print(f"   Year range: {df_vv['year'].min():.0f} - {df_vv['year'].max():.0f}")
df_vv = df_vv.sort_values(by=['rating'], ascending=False)
display(df_vv.head(20))



📊 Vivino Dataset Overview:
   Total wines: 11,298
   Columns: ['winery', 'year', 'wine_id', 'description', 'rating', 'reviews', 'price', 'region', 'variety', 'natural', 'country', 'type', 'country_code', 'rank']
   Price range: $2 - $6511
   Varietal: ['Malbec', 'Shiraz/Syrah', 'Cabernet Sauvignon', 'Riesling', 'Tempranillo', 'Pinot Noir', 'Sangiovese', 'Touriga Nacional']
   Year range: 1900 - 2021


,winery,year,wine_id,description,rating,reviews,price,region,variety,natural,country,type,country_code,rank
9079,Quinta do Noval,1994,81431,Vintage Port Nacional 1994,4.9,35,2784.65,Porto,Touriga Nacional,False,Portugal,fortified,PRT,1.000000
9802,Niepoort,1900,8971,Colheita Port 1900,4.9,39,1870.00,Porto,Touriga Nacional,False,Portugal,fortified,PRT,1.000000
111,Catena Zapata,2018,4835815,Adrianna Vineyard River Stones Malbec 2018,4.8,40,149.04,Mendoza,Malbec,False,Argentina,red,ARG,0.956522
6230,Château Cheval Blanc,2017,1148747,Saint-Émilion Grand Cru (Premier Grand Cru Cla...,4.8,60,763.50,Saint-Émilion Grand Cru,Pinot Noir,False,France,red,FRA,0.956522
9433,Ferreira,2018,75996,Vintage Port 2018,4.8,115,99.00,Porto,Touriga Nacional,False,Portugal,fortified,PRT,0.956522
8612,Quinta do Noval,2001,81431,Vintage Port Nacional 2001,4.8,49,1692.25,Porto,Touriga Nacional,False,Portugal,fortified,PRT,0.956522
10447,Promontory,2014,3985191,Promontory 2014,4.8,150,1029.20,Valle de Napa,Cabernet Sauvignon,False,United States,red,USA,0.956522
10029,Screaming Eagle,2012,82025,Cabernet Sauvignon 2012,4.8,234,6511.31,Oakville,Cabernet Sauvignon,False,United States,red,USA,0.956522
11919,Avondale,2008,2747664,Navitas 2008,4.8,44,175.00,Paarl,Cabernet Sauvignon,False,South Africa,red,ZAF,0.956522
10296,Hundred Acre,2012,1637962,Ark Vineyard Cabernet Sauvignon 2012,4.8,294,798.00,Howell Mountain,Cabernet Sauvignon,False,United States,red,USA,0.956522


In [ ]:
df = pd.concat([df_vv, df], axis=0)


df = df.dropna(subset=['description', 'variety', 'year', 'price'])


df = df[(df['year'] >= vintage_min) & (df['year'] <= vintage_max)]

# Basic exploration
print(f"\n📊 Dataset Overview:")
print(f"   Total wines: {len(df):,}")
print(f"   Columns: {list(df.columns)}")
print(f"   Price range: ${df['price'].min():.0f} - ${df['price'].max():.0f}")
# print(f"   Regions: {list(df['province'].drop_duplicates())}")
print(f"   Varietal: {list(df['variety'].drop_duplicates())}")
print(f"   Year range: {df['year'].min():.0f} - {df['year'].max():.0f}")


# Create a scatter plot of the data. To change the markers to red "x",
# we used the 'marker' and 'c' parameters
df = df[df['price'] <= 100]
df = df[df['type'] == 'red'].copy()
top_20 = df['variety'].value_counts().head(20)
df = df[df['variety'].isin(top_20.index.tolist())]

df = df[df['rating'] >= 0]

df = df.sort_values(by=['rating'], ascending=False)

display(df.head(20))


# PLOT DATA
# plt.scatter(df['price'], df['rank'], marker='x', c='r') 

# # Set the title
# plt.title("rank vs. Price")
# # Set the y-axis label
# plt.ylabel('Y')
# # Set the x-axis label
# plt.xlabel('X')
# plt.show()

# Show sample wines
print(f"\n🍾 Sample Wines:")
# display(df.head())

# LINEAR REG PARAMS

# features = ['province', 'vintage', 'variety'] # ['province', 'vintage', 'country', 'variety'] 

# do the reg only for most popular reds
df = df[df['type'] == 'red'].copy()
top_20 = df['variety'].value_counts().head(20)
df = df[df['variety'].isin(top_20.index.tolist())]

target = 'price'
features = ['year', 'variety']

# remove outlier for y (price)
df = df[df[target] <= 100]

# Drop first helps with multicollinearity, i.e. when 2 features are correlated and their impact on y cannot be detangled
df = pd.get_dummies(df, columns=['variety'], drop_first=True)

# df = pd.get_dummies(df, columns=['province'], drop_first=True)

# For the year, we create 2x features, one for the age (normalized 0..1) and one for the vintage (one-hot)
scaler = StandardScaler()
df['age'] = vintage_max - df['year']
df['age_normalized'] = scaler.fit_transform(df[['age']])

df = pd.get_dummies(df, columns=['year'], drop_first=True)



# points


encoded_features = [col for col in df.columns 
                   if any(col.startswith(prefix + '_') for prefix in features)]


X = df[encoded_features + ['age_normalized', 'rank']] # 
y = df[target]


print(X.shape)


print(f"   Total wines: {len(X):,}")
print(f"   Features: {list(X.columns)}")
print(f"   Price range: ${y.min():.0f} - ${y.max():.0f}")
print(f"   Age range: {df['age'].min():.0f} - {df['age'].max():.0f}")


# 2. Split the data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 3. Create and train the model
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(X_train, y_train)

# # 4. Make predictions
# y_pred = model.predict(X_test)

# # 5. Evaluate
# from sklearn.metrics import r2_score, mean_squared_error
# r2 = r2_score(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))


# What this means for your results:
# RMSE = 25.056:

# Direct measure of cost function performance
# "On average, predictions are off by $25"

# R² = 0.038:

# Your model's error vs baseline error
# "Your model is only 3.8% better than just predicting the mean price ($40.46) every time"
# print(f"R² Score: {r2:.3f}")
# print(f"RMSE: {rmse:.3f}")

# print(f"Target range: {y.min()} to {y.max()}")
# print(f"Target mean: {y.mean():.2f}")
# print(f"Number of features: {X.shape[1]}")
# print(f"Training samples: {X.shape[0]}")


# Search for wine/winery
# df = df[df['title'].str.contains('sister', case=False, na=False)]
# df = df[df['title'].str.contains('run', case=False, na=False)]
# df = df[df['province'].str.contains('Australia Other', case=False, na=False)]

# display(df.head(30))




📊 Dataset Overview:
   Total wines: 13,572
   Columns: ['winery', 'year', 'wine_id', 'description', 'rating', 'reviews', 'price', 'region', 'variety', 'natural', 'country', 'type', 'country_code', 'rank', 'Unnamed: 0', 'designation', 'points', 'province', 'region_1', 'region_2', 'title']
   Price range: $2 - $6511
   Varietal: ['Touriga Nacional', 'Malbec', 'Pinot Noir', 'Cabernet Sauvignon', 'Sangiovese', 'Shiraz/Syrah', 'Riesling', 'Tempranillo', 'pinot noir', 'shiraz-cabernet sauvignon', 'shiraz', 'shiraz-viognier', 'viognier', 'riesling', 'cabernet sauvignon', 'red blend', 'chardonnay', 'marsanne', 'cabernet-shiraz', 'grenache', 'cabernet sauvignon-shiraz', 'moscato', 'sémillon', 'rosé', 'rhône-style red blend', 'syrah', 'g-s-m', 'pinot gris', 'bordeaux-style red blend', 'shiraz-grenache', 'sangiovese', 'cabernet sauvignon-merlot', 'sauvignon blanc', 'champagne blend', 'petite sirah', 'port', 'shiraz-cabernet', 'grenache-shiraz', 'sparkling blend', 'roussanne', 'cabernet sauvignon

,winery,year,wine_id,description,rating,reviews,price,region,variety,natural,...,type,country_code,rank,Unnamed: 0,designation,points,province,region_1,region_2,title
11759,Saxenburg,2015,1521195.0,Select Shiraz 2015,4.8,31.0,89.00,Stellenbosch,Cabernet Sauvignon,False,...,red,ZAF,0.956522,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9695,Wine & Soul,2017,1253819.0,Douro Pintas Tinto 2017,4.7,189.0,69.90,Douro,Touriga Nacional,False,...,red,PRT,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,El Enemigo,2017,3125433.0,Gran Enemigo Single Vineyard Gualtallary Caber...,4.7,310.0,93.00,Gualtallary,Malbec,False,...,red,ARG,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8851,Niepoort,2012,2899486.0,Douro Turris 2012,4.7,101.0,99.00,Douro,Touriga Nacional,False,...,red,PRT,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,Catena Zapata,2018,1869.0,Malbec Argentino 2018,4.7,1613.0,85.00,Mendoza,Malbec,False,...,red,ARG,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11346,Annandale,2004,5425213.0,The Key Merlot 2004,4.7,87.0,75.00,Stellenbosch,Cabernet Sauvignon,False,...,red,ZAF,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,El Enemigo,2017,5441683.0,Gran Enemigo Single Vineyard El Cepillo Cabern...,4.7,44.0,69.93,San Carlos,Malbec,False,...,red,ARG,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9477,Quinta Vale D. Maria,2018,1896436.0,Vinha da Francisca Douro 2018,4.7,169.0,68.93,Douro,Touriga Nacional,False,...,red,PRT,0.913043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,Catena Zapata,2018,1868.0,Nicolás Catena Zapata 2018,4.6,69.0,71.00,Mendoza,Malbec,False,...,red,ARG,0.869565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10394,Law Estate Wines,2012,3925645.0,Sagacious 2012,4.6,76.0,71.67,Paso Robles,Cabernet Sauvignon,False,...,red,USA,0.869565,NaN,NaN,NaN,NaN,NaN,NaN,NaN



🍾 Sample Wines:
(6144, 38)
   Total wines: 6,144
   Features: ['variety_Malbec', 'variety_Pinot Noir', 'variety_Riesling', 'variety_Sangiovese', 'variety_Shiraz/Syrah', 'variety_Tempranillo', 'variety_Touriga Nacional', 'year_1993', 'year_1994', 'year_1995', 'year_1996', 'year_1997', 'year_1998', 'year_1999', 'year_2000', 'year_2001', 'year_2002', 'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'age_normalized', 'rank']
   Price range: $2 - $100
   Age range: 4 - 40


In [11]:

# Create correlation matrix for your features
# correlation_matrix = X.corr()
# mask = np.abs(correlation_matrix) < 0.5  # Hide weak correlations

# # Plot the heatmap
# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm')
# plt.title('Strong Correlations Only (>0.5)')
# plt.show()

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.regularizers import L2 

# convert bools to 0/1
cs = X.select_dtypes(include=['bool']).columns
X[cs] = X[cs].astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Price range: ${y.min():.0f} to ${y.max():.0f}")
print(f"Average price: ${y.mean():.0f}")
print(f"Price std dev: ${y.std():.0f}")


Y_train = np.array(y_train)
X_train = X_train.to_numpy()

Y_test = np.array(y_test)
X_test = X_test.to_numpy()
display(X_test)




# Build the model, 88 features
# model = Sequential([
#     Dense(100, activation='relu', input_shape=(38,)),  # Hidden layer
#     Dense(50, activation='relu'),  # Hidden layer
#     Dense(20, activation='relu'),  # Hidden layer
#     Dense(1, activation='linear')                     # Output layer
# ])

model = Sequential([
    Dense(500, activation='relu', input_shape=(38,)),
    Dense(200, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(50, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(20, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(1, activation='linear', kernel_regularizer=L2(0.01))
])


#print(X_train)
# print(Y_train)

model.compile(loss=MeanSquaredError, metrics=['mae', 'mse'])
model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

test_loss, test_mae, test_mse = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MSE: {test_mse:.4f}")



Price range: $2 to $100
Average price: $25
Price std dev: $21


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.81117488,  0.47826087],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.47111904,  0.47826087],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         1.2291602 ,  0.56521739],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
        -0.47111904,  0.47826087],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.5490485 ,  0.82608696],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.13106319,  0.56521739]])

Epoch 1/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 483.7198 - mae: 14.9252 - mse: 480.0356 - val_loss: 240.4683 - val_mae: 10.0613 - val_mse: 237.3746
Epoch 2/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 247.3144 - mae: 10.7475 - mse: 244.2004 - val_loss: 196.0710 - val_mae: 9.1208 - val_mse: 192.8678
Epoch 3/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 191.1298 - mae: 9.4271 - mse: 187.8823 - val_loss: 200.8545 - val_mae: 8.5899 - val_mse: 197.4660
Epoch 4/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 188.4757 - mae: 8.9806 - mse: 185.0564 - val_loss: 181.7294 - val_mae: 8.6198 - val_mse: 178.2408
Epoch 5/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 175.8345 - mae: 8.6749 - mse: 172.3331 - val_loss: 179.5156 - val_mae: 8.5856 - val_mse: 175.9738
Epoch 6/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 174.0688 - mae: 8.6297 - mse: 170.5190 - val_loss: 178.8027 - val_mae: 8.5108 - val_mse: 175.2512
Epoch 7/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

In [8]:

# print(Y_test.shape)
# print(X_test.shape)

# for i in range(Y_test.shape[0]):
#     print(X_test[i,:].shape)
#     p = model.predict(X_test[i,:])
#     print(p)